<a href="https://colab.research.google.com/github/Addechander/fcc-cats-and-dogs-image-classifier/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-08-13 17:20:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  35.2MB/s    in 0.7s    

2023-08-13 17:20:44 (35.2 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.isna().any()

isbn      False
title     False
author     True
dtype: bool

In [ ]:
df_ratings.isna().any()

user      False
isbn      False
rating    False
dtype: bool

In [ ]:
df_ratings.columns

Index(['user', 'isbn', 'rating'], dtype='object')

In [ ]:
user_filter = df_ratings['user'].value_counts()
isbn_filter = df_ratings['isbn'].value_counts()

user_filter

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user, Length: 105283, dtype: int64

In [ ]:
isbn_filter

0971880107     2502
0316666343     1295
0385504209      883
0060928336      732
0312195516      723
               ... 
1568656386        1
1568656408        1
1569551553        1
1570081808        1
05162443314       1
Name: isbn, Length: 340556, dtype: int64

In [ ]:
df_ratings = df_ratings[df_ratings['isbn'].isin(isbn_filter.index) & df_ratings['user'].isin(user_filter.index)]

In [ ]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [ ]:
df_merged = pd.merge(df_ratings, df_books, on = 'isbn')

In [ ]:
df_merged.isna().any()

user      False
isbn      False
rating    False
title     False
author     True
dtype: bool

In [ ]:
df_merged.dropna(inplace = True)

In [ ]:
df_merged.shape

(1031174, 5)

In [ ]:
df_merged.columns

Index(['user', 'isbn', 'rating', 'title', 'author'], dtype='object')

In [ ]:
df_merged_sample = df_merged.sample(frac=.1, random_state=1)

df_merged_sample.shape

(103117, 5)

In [ ]:
df_merged_sample

,user,isbn,rating,title,author
839442,87825,0140186417,0.0,In Dubious Battle (20th Century Classics),John Steinbeck
320829,5416,0425142035,5.0,Missing in Manhattan: The Adams Round Table,Bill Adler
456519,142801,0345335511,0.0,"Pawn of Prophecy (The Belgariad, Book 1)",David Eddings
888113,152018,1561380040,8.0,Gentle Closings: How to Say Goodbye to Someone...,Ted Menten
69970,269566,0425161242,0.0,Chromosome 6,Robin Cook
...,...,...,...,...,...
968838,187520,0140512101,9.0,Penguin Dictionary of Art and Artists (Penguin...,Peter Murray
121248,269781,0373218192,7.0,Engaging The Enemy,Nora Roberts
870594,171295,0060175982,3.0,Mars and Venus Starting Over: A Practical Guid...,John Gray
763576,55492,0671003879,0.0,Savage Wilderness,Harold Coyle


In [ ]:
df_merged_pivot = df_merged_sample.pivot_table(index = 'isbn', columns='user', values = 'rating').fillna(0)

In [ ]:
mat_book_features = csr_matrix(df_merged_pivot.values)

In [ ]:
df_merged_pivot

user,8,9,12,14,20,53,69,83,87,99,...,278721,278732,278771,278832,278843,278846,278849,278851,278852,278854
isbn,,,,,,,,,,,,,,,,,,,,,
0001711105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001841572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001952404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002000202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002118580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0000A2U93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0000AZW79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00014WRH4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_merged_pivot.index = df_merged_pivot.join(df_books.set_index('isbn'))['title']
df_merged_pivot

user,8,9,12,14,20,53,69,83,87,99,...,278721,278732,278771,278832,278843,278846,278849,278851,278852,278854
title,,,,,,,,,,,,,,,,,,,,,
The Book of Riddles (Beginner Books),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Red Shift,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Friend Monkey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Black Berry, Sweet Juice: On Being Black and White in Canada",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Audacity to believe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Carmilla,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O Is for Outlaw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hornet Flight: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = NearestNeighbors(metric='cosine')

In [ ]:
model.fit(mat_book_features)

NearestNeighbors(metric='cosine')

In [ ]:
df_merged_sample

,user,isbn,rating,title,author
296269,225659,2253002879,8.0,Le Noeud de Viperes,F. Mauriac
62847,142026,0385315287,8.0,Running from Safety: An Adventure of the Spirit,Richard Bach
305959,108812,0375421556,8.0,Red Dog,LOUIS DE BERNIERES
20731,177091,0451203771,6.0,Scarlet Feather,Maeve Binchy
374898,238630,0930452798,0.0,The Road to Mecca,Athol Fugard
...,...,...,...,...,...
208964,188091,3453210697,5.0,Die Bruderschaft,John Grisham
119969,9414,0140351140,8.0,Around the World in Eighty Days (Puffin Classics),Jules Verne
17426,278663,0061097314,4.0,The Bean Trees,Barbara Kingsolver
346830,243964,0373710550,0.0,Fleet Hospital (In Uniform) (Harlequin Superro...,Anne Marie Duquette


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

    distance, indices=model.kneighbors(df_merged_pivot.loc[[book]], 6, return_distance=True)

    distance = distance[0][1:]
    indices = indices[0][1:]

    recommended_books = []
    for value in (df_merged_pivot.index[indices]):
      recommended_books.append(value)

    recommended = [list(z) for z in zip(recommended_books, distance)][::-1]
    return [book['title'].values[0], recommended]



books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)
print(books[1][1][1])

KeyError: ignored

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Running in the Family (Vintage International)', 1.0], ['Hotel Pastis : A Novel of Provence', 1.0], ['Mama Makes Up Her Mind: And Other Dangers of Southern Living', 1.0], ['Dead Man Walking: An Eyewitness Account of the Death Penalty in the United States', 1.0], ['Sanctuary: The Corrected Text', 1.0]]]
You havn't passed yet. Keep trying!
